In [1]:
pip install -U langchain-community tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successful

In [4]:
import os
#import getpass
from google.colab import userdata
#from llamaindex import Document, GPTIndex, SimpleRetriever

# Retrieve your Google API key
tavily_api_key = userdata.get('TAVILY_API_KEY')
os.environ["TAVILY_API_KEY"] = tavily_api_key
openai_api_key=userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key

#os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [5]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

retriever.invoke("what year was breath of the wild released?")

[Document(metadata={'title': 'The Legend of Zelda: Breath of the Wild - Nintendo Switch Wiki', 'source': 'https://nintendo-switch.fandom.com/wiki/The_Legend_of_Zelda:_Breath_of_the_Wild', 'score': 0.9955316, 'images': []}, page_content='The Legend of Zelda: Breath of the Wild is an open world action-adventure game published by Nintendo for the Wii U and as a launch title for the Nintendo Switch, and was released worldwide on March 3, 2017.It is the nineteenth installment of the The Legend of Zelda series and the first to be developed with a HD resolution. The game features a gigantic open world, with the player being able to ...'),
 Document(metadata={'title': 'The Legend of Zelda: Breath of the Wild - Zelda Wiki', 'source': 'https://zelda.fandom.com/wiki/The_Legend_of_Zelda:_Breath_of_the_Wild', 'score': 0.9804313, 'images': []}, page_content='[]\nReferences\nThe Legend of Zelda \xa0·\nThe Adventure of Link \xa0·\nA Link to the Past (& Four Swords) \xa0·\nLink\'s Awakening (DX; Ninten

In [6]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 13.5 MB/s eta 0:00:00


In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """You're an expert Dermatologist. Answer the question based only on the context provided. If the question asks to List then format the answer in bullets.

Context: {context}

Question: {question}"""
)
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | ChatOpenAI(model="gpt-4-1106-preview", openai_api_key=openai_api_key)
    | StrOutputParser()
)

In [8]:
chain.invoke({"question": "how many units did bretch of the wild sell in 2020"})

'- The Legend of Zelda: Breath of the Wild sold over 20.1 million copies worldwide as of August 6, 2020.'

In [9]:
chain.invoke({"question": "Whata are the causes of skin cancer?"})

'- Overexposure to ultraviolet (UV) light, either from the sun or from artificial sources like tanning beds.\n- Other important causes of skin cancer (not specified in the context provided, but generally include factors such as genetic predisposition, certain types of human papillomavirus, exposure to certain chemicals, and a weakened immune system).'

In [10]:
chain.invoke({"question": "List and describe the common treatments of Eczema?"})

"- **Emollients**: These are moisturizing treatments applied directly to the skin to reduce water loss and cover it with a protective film. They're often used to help manage dry, itchy, or scaly skin conditions such as eczema. Regular use of emollients is the mainstay of treatment for all stages and severities of eczema.\n\n- **Topical Steroids**: These are anti-inflammatory medicines used to treat a range of conditions affecting the skin. They can help to reduce inflammation, itching, and redness associated with eczema. Topical steroids come in various strengths and formulations, and their use must be carefully managed to avoid side effects.\n\n- **Paste Bandages and Wet Wraps**: These are special bandages or garments that can be applied over emollients or medicated creams. They are particularly useful for areas of eczema that are difficult to manage, help to hydrate the skin, reduce itching, and provide a physical barrier against scratching.\n\n- **Sedating-type Antihistamines**: The

In [11]:
chain.invoke({"question": "What is Acne? What should I do if have acne?"})

"Acne is a skin condition that occurs when your hair follicles become plugged with oil and dead skin cells. It typically causes whiteheads, blackheads, or pimples and is most common among teenagers, though it can affect people of all ages.\n\nIf you have acne, you can:\n\n- Start an at-home skin care routine which may include:\n  - Cleansing your skin gently with a mild, non-abrasive cleanser.\n  - Using over-the-counter acne treatments containing ingredients like benzoyl peroxide.\n  - Applying the treatment as directed, allowing time for the product to penetrate the skin before rinsing off, if applicable.\n  - Avoiding picking or squeezing pimples, which can worsen acne and cause scarring.\n\n- Visit a healthcare provider if:\n  - Your at-home treatments aren't effective.\n  - You have severe or persistent acne.\n  - You have sudden, severe outbreaks, especially if you're an adult, which might require further evaluation for underlying conditions.\n\nThe healthcare provider may recomm